In [1]:
import sqlite3
import psycopg2
import pandas as pd
from DataBase_functions import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [2]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    # Seleccionar la hoja específica por nombre
    sheet_name = input("Choose the sheet: ")  # Reemplaza con el nombre de tu hoja
    sheet = spreadsheet.worksheet(sheet_name)
    # Obtener los encabezados esperados
    expected_headers = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]

    # Obtener todos los registros de la hoja
    data = sheet.get_all_records(expected_headers=expected_headers)
    df = pd.DataFrame(data)

    # Mostrar las primeras filas del DataFrame
    print(df.head())

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Julio', 'Junio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Responsables']
  Fecha de alta Fecha incurrida Fecha de fin UUAAA        Código SDATOOL  \
0    24/09/2024      01/10/2024   01/10/2024  KESJ  DQ-DP-007019   45403   
1    24/09/2024      01/10/2024   01/10/2024  KESJ  DQ-DP-007019   45403   
2    27/09/2024      01/10/2024   01/10/2024  FACC  DQ-DP-007037   34644   
3    27/09/2024      01/10/2024   01/10/2024  KHFC                         
4    30/09/2024      01/10/2024   01/10/2024  MCRR  DQ-DP-007041   38358   

          Feature         Petición Geografía     Gestor BBDD   Ámbito  \
0  AUDLEGGES4-249  REQ000101515343    España       PosgreSQL   Legacy   
1  AUDLEGGES4-249  REQ000101515343    España       PosgreSQL   Legacy   
2         ADH-678     OTRSSD-17407    España  Oracle Physics  NextGEN

In [6]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

 
#Cargar credenciales
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

#Autorizar gspread con las credenciales
client = gspread.authorize(creds)

# ID de la hoja de cálculo 
srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

# Open the Google Sheet file by ID
spreadsheet = client.open_by_key(srpeadsheet_id)

# List all sheets file
sheets = spreadsheet.worksheets()
print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

# Seleccionar la hoja específica por nombre
sheet_name = input("Choose the sheet: ")  # Reemplaza con el nombre de tu hoja
sheet = spreadsheet.worksheet(sheet_name)
# Obtener los encabezados esperados
expected_headers = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]

# Obtener todos los registros de la hoja
data = sheet.get_all_records(expected_headers=expected_headers)
df = pd.DataFrame(data)

# Mostrar las primeras filas del DataFrame
print(df.head())

RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})